INTRO

In [37]:
import eumdac
from eumdac.datatailor import Filter, RegionOfInterest
import json
import os
import datetime
import time
import numpy as np
import fnmatch
import shutil
import glob
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.core.display import HTML

import warnings
warnings.filterwarnings('ignore')

OPTS

In [2]:
run_custom = True

AUTH

In [3]:
# load credentials
with open(os.path.join(os.path.expanduser("~"),'.eumdac_credentials')) as json_file:
    credentials = json.load(json_file)
    token = eumdac.AccessToken((credentials['consumer_key'], credentials['consumer_secret']))
    print(f"This token '{token}' expires {token.expiration}")

# create data store object
datastore = eumdac.DataStore(token)

This token '72792db1-a3f9-3081-9a5b-ad81d222a5d7' expires 2022-06-07 14:51:41.584779


SEARCH

In [4]:
collection_id = 'EO:EUM:DAT:MSG:HRSEVIRI'
selected_collection = datastore.get_collection(collection_id)
start = datetime.datetime(2022, 6, 3, 7, 0)
end = datetime.datetime(2022, 6, 3, 19, 0)
products = selected_collection.search(dtstart=start, dtend=end)

PRUNE

In [5]:
selected_products = []
for product, count in zip(products, range(len(products))):
    if np.mod(count, 4) == 0: 
        selected_products.append(product)

TAILOR

In [6]:
datatailor = eumdac.DataTailor(token) 

CHAIN

In [7]:
my_chain = eumdac.tailor_models.Chain(
            id='UK-coccos',
            name='UK-coccos',
            description='Native to NetCDF of Bristol Channel',
            product='HRSEVIRI',
            format='netcdf4',
            filter=Filter(id=None, bands=['channel_1'], name=None, product=None),
            projection='geographic',
            roi=RegionOfInterest(id=None, name=None, NSWE=[52, 49.5, -6.5, -2], description=None)
    )

CUSTOMISE

In [12]:
# this is not very smart, but works as long as nothing goes wrong on the DT side
if run_custom:
    print("Go make a cup of tea, this may take a little bit of time...")
    for product, count in zip(selected_products, range(len(selected_products))):
        # launch
        customisation = datatailor.new_customisation(product, chain=my_chain)
        print(f"({customisation._id}): Launched ({count+1}/{len(selected_products)})")
        # wait til done
        while customisation.status != "DONE":
            print(f"({customisation._id}): Status: {customisation.status}")
            time.sleep(5)
        print(f"({customisation._id}): Status: {customisation.status}")
        # download
        netcdf_file, = fnmatch.filter(customisation.outputs, '*.nc')
        with customisation.stream_output(netcdf_file,) as stream, \
          open(stream.name, mode='wb') as fdst:
            shutil.copyfileobj(stream, fdst)
        print(f"({customisation._id}): Downloaded the NetCDF output")
        # clear
        customisation.delete()
        print(f"({customisation._id}): Cleared the customisation \n ---")
else:
    print("Skipping customisation")

Go make a cup of tea, this may take a little bit of time...
(ea467924): Launched (1/16)
(ea467924): Status: QUEUED


KeyboardInterrupt: 

LOAD DATA

In [69]:
seviri_data = xr.open_mfdataset(glob.glob('*.nc'), concat_dim='t', combine='nested')
data = np.asarray(seviri_data.channel_1)
data[data < 1e-3] = 1e-3
data[data > 2] = 2
data = np.log10(data)

BUILD ANIMATION

In [70]:
%%capture
# build  quick animation; thanks to https://brushingupscience.com/2016/06/21/matplotlib-animations-the-easy-way/

# initialise plot
fig = plt.figure(figsize=(20,10))
pad = 1
ax = plt.subplot(1, 1, 1)

# initialise data
ax.pcolor(seviri_data.lon, seviri_data.lat, data[0,:,:], cmap=plt.cm.Spectral_r)

# build animation function; clearing axes removes previous plot as quiver cannot natively update x/y, only UVC.
def animate(i):
    for c in ax.collections:
        c.remove()
    ax.pcolor(seviri_data.lon, seviri_data.lat, data[i,:,:], cmap=plt.cm.Spectral_r)

# animate
anim = FuncAnimation(fig, animate, interval=1000, frames=np.shape(seviri_data.channel_1)[0]-1)

RUN ANIMATION

In [71]:
HTML(anim.to_jshtml())